# Tutorial: Peralatan Dasar untuk Privasi Pembelajaran Dalam (*Private Deep Learning*)

Selamat datang di tutorial pengantar PySft untuk menjaga privasi, pembelajaran dalam terdesentralisasi. Rangkaian buku catatan ini (Jupyter notebooks) adalah panduan tahap demi tahap agar anda dapat mengetahui peralatan dan teknik baru yang diperlukan untuk menjalankan pembelajaran dalam pada model/data privat/rahasia tanpa tersentralisasi dalam otoritas tunggal.

**Cakupan:** Perhatikan bahwa kita tidak akan hanya membahas tentang bagaimana mendesentralisasi atau mengenkripsi data, tapi kita juga akan menekankan bagaimana PySyft akan digunakan untuk membantu desentralisasi dari keseluruhan ekosistem seputar data, bahkan termasuk basis data (*database*) dimana data disimpan dan dikueri, dan model saraf tiruan (*neural models*) yang digunakan untuk mengekstrak informasi dari data. Dengan ekstensi-ekstensi PySyft yang baru telah dibuat, buku catatan ini akan ditambahkan dengan tutorial-tutorial baru untuk menjelaskan fungsionalitas yang baru.

Pengarang:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Penerjemah:
- Linggih Saputro - Twitter: [@__nggih](https://twitter.com/__nggih)

## Ikhtisar:

- Bagian 1: Peralatan Dasar untuk Privasi Pembelajaran Dalam

## Mengapa Perlu Mengambil Tutorial Ini?

**1) Karir dengan Keunggulan Kompetitif** - Dalam 20 tahun belakangan ini, revolusi digital telah mempermudah data untuk dapat diakses dalam kuantitas yang besar seiring dengan proses-proses analog yang telah didigitalisasi. Akan tetapi, dengan peraturan baru seperti [GDPR](https://eugdpr.org/), perusahaan-perusahan ditekan agar tidak leluasa menggunakan data mereka - dan yang lebih penting lagi, bagaimana mereka menganalisis informasi personal. **Garis Bawah:** Ilmuwan data tidak akan memiliki akses terhadap data dengan peralatan "kuno", tapi dengan mempelajari peralatan Privasi Pembelajaran Dalam, ANDA bisa lebih maju dan memperoleh keunggulan kompetitif di karir anda.

**2) Peluang Bisnis** - Terdapat begitu banyak permasalahan di masyarakat yang dapat diselesaikan dengan Pembelajaran Dalam, tapi banyak dari permasalahan yang paling penting belum dapat dieksplorasi karena dibutuhkan akses terhadap informasi yang sangat sensitif milik banyak orang (pertimbangkan penggunaan Pembelajaran Dalam untuk membantu orang dengan masalah mental atau hubungan!). Maka dari itu, belajar Privasi Pembelajaran Dalam membuka begitu banyak peluang perusahaan rintisan (*start-up*) baru untuk anda yang sebelumnya tidak tersedia tanpa peralatan ini.

**3) Kebaikan Sosial** - Pembelajaran Dalam dapat digunakan untuk menyelesaikan bermacam-macam masalah di dunia nyata, tapi Pembelajaran Dalam akan *informasi personal* adalah Pembelajaran Dalam tentang orang-orang, *untuk orang-orang*. Belajar bagaimana melakukan Pembelajaran Dalam pada data yang tidak anda miliki, menunjukkan lebih dari sekedar peluang karir atau bisnis, ini adalah peluang untuk membantu menyelesaikan beberapa dari masalah-masalah personal dan penting dalam hidup orang-orang - dan melakukannya dalam skala besar.


## Bagaimana saya dapat memperoleh kredit ekstra?
- Bintangi PySyft di GitHub! -[https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Mengajarkan buku catatan (*jupyter notebook) ini lewat video Youtube!

... oke ... mari kita mulai!

# Part -1: Prasyarat
- Mengenal PyTorch - jika tidak, maka pelajari kuliah http://fast.ai dahulu kemudian kembali lagi kemari.
- Bacalah jurnal PySyft Framework https://arxiv.org/pdf/1811.04017.pdf! Ini akan memberikan anda latar belakang yang menyeluruh tentang bagaimana PySyft dibentuk, yang akan menjadikan hal-hal ini lebih masuk akal.

# Part 0: Setup

Untuk memulai, anda harus memastikan instalasi anda benarr. Bacalah PySyft bagian README di laman PySyft dan ikuti instruksi instalasi.

- Pasang Python 3.6 atau versi lebih tinggi
- Pasang PyTorch 1.3
- Klon (*clone*) PySyft (git clone https://github.com/OpenMined/PySyft.git)
- cd PySyft
- pip install -r pip-dep/requirements.txt
- pip install -r pip-dep/requirements_udacity.txt
- python setup.py install udacity
- python setup.py test

Jika ada bagian yang tidak jalan (atau tes apapun yang gagal) - periksa terlebih dahulu [README](https://github.com/OpenMined/PySyft.git) untuk bantuan instalasi lalu buka GitHub Issue atau kabari kanal #beginner di Slack kami! [slack.openmined.org](http://slack.openmined.org/)

In [2]:
# Jalankan sel ini untuk memastikan semuanya berjalan dengan baik (Shift+Enter)
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

Jika sel di atas dijalankan, maka anda dapat memulai yang selanjutnya! Mari kita lanjutkan!

# Bagian 1: Peralatan untuk Ilmu Data yang Terdesentralisasi dan Privat

Jadi - pertanyaan pertama anda mungkin - Bagaimana kita dapat melatih model di data tanpa kita dapat akses?

Jawabannya mungkin sangat sederhana. Jika anda terbiasa bekerja dengan PyTorch, maka anda juga terbiasa bekerja dengan objek-objek torch.Tensor seperti ini!

In [4]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


Jelas, sangat penting menggunakan tensor yang super lucu dan kuat ini, tapi juga mengharuskan anda memiliki data di mesin lokal anda. Dari sinilah perjalanan anda dimulai.

# Bab 1.1 - Mengirimkan Tensor ke Mesinnya Bob

Sedangkan biasanya kita melakukan ilmu data/ Pembelajaran Dalam di mesin yang terdapat data tersebut, sekarang, kita ingin melakukan komputasi di mesin **lain**. Terlebih secara khusus, kita tidak dapat berasumsi data tersebut terdapat di mesin lokal kita.

Dengan demikian, alih-alih menggunakan tensor Torch, kita akan bekerja dengan **pointer** ke tensor. Izinkan saya tunjukkan yang saya maksud. Pertama, marilah kita buat mesin "bohongan" yang dimiliki oleh orang "bohongan" - sebut saja namanya Bob.

In [37]:
bob = sy.VirtualWorker(hook, id="bob")

Katakanlah mesinnya Bob ada di planet lain - mungkin saja di Mars! Tapi, sekarang ini mesinnya kosong. Mari kita buat beberapa data agar kita dapat mengirimnya ke Bob dan belajar tentang *pointer*!

In [38]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

Dan sekarang - marilah kita kirim tensor kita ke Bob!!

In [39]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [40]:
x_ptr

(Wrapper)>[PointerTensor | me:43091033991 -> bob:80505619433]

DOR! Sekarang si Bob punya dua tensor! Ga percaya? Kita buktikan bersama!

In [41]:
bob._objects

{80505619433: tensor([1, 2, 3, 4, 5]), 16737735861: tensor([1, 1, 1, 1, 1])}

In [42]:
z = x_ptr + x_ptr

In [43]:
z

(Wrapper)>[PointerTensor | me:30357512505 -> bob:36076025945]

In [44]:
bob._objects

{80505619433: tensor([1, 2, 3, 4, 5]),
 16737735861: tensor([1, 1, 1, 1, 1]),
 36076025945: tensor([ 2,  4,  6,  8, 10])}

Sekarang dapat kita perhatikan. Ketika kita panggil `x.send(bob)`, itu mengembalikan objek baru yang kita sebut dengan `x_ptr`. Ini adalah *pointer* pertama kita untuk sebuah tensor. *Pointer* ke tensor TIDAK menyimpan data itu sendiri. Melainkan, mereka menyimpan metadata tentang tensor (dengan data) disimpan di mesin lainnnya. Tujuan dari tensor-tensor ini adalah untuk menyediakan API yang intuitif untuk memberitahukan mesin yang lainnya untuk menghitung fungsi-fungsi menggunakan tensor ini. Mari kita lihat metadata yang disimpan *pointer-pointer* tersebut.


In [45]:
x_ptr

(Wrapper)>[PointerTensor | me:43091033991 -> bob:80505619433]

Mari kita periksa metadata tersebut!

Terdapat dua atribut utama yang spesifik terhadap *pointer-pointer* tersebut:

- `x_ptr.location : bob`, sebuah referensi ke lokasi yang ditunjuk oleh *pointer* 
- `x_ptr.id_at_location : <random integer>`, dengan tipe data bilangan bulat yang acak, menyediakan identitas (id) lokasi tensor itu disimpan.

Mereka dicetak dengan format `<id_at_location>@<location>`

Terdapat juga atribut umum lainnya:
- `x_ptr.id : <random integer>`, dengan tipe data bilangan, identitas (id) dari tensor *pointer* kita, yang dialokasikan secara acak.

- `x_ptr.owner : "me"`, pekerja (*worker*) yang memiliki tensor *pointer*, dalam contoh ini, pekerja (*worker*) lokal, yang dinamakan "me"


In [46]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [47]:
bob

<VirtualWorker id:bob #objects:3>

In [48]:
bob == x_ptr.location

True

In [49]:
x_ptr.id_at_location

80505619433

In [50]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

Anda penasaran 'kan kenapa pekerja (*worker*) lokal yang memiliki *pointer* juga adalah sebuah VirtualWorker, walaupun kita tidak membuatnya.
Fakta lucu nih, sama seperti objek VirtualWorker untuk Bob, kita juga punya satu untuk kita (secara standar). Pekerja (*worker*) ini secara otomatis dibuat ketika kita memanggil `hook = sy.TorchHook()`, sehingga kita tidak perlu membuatnya sendiri. 

In [51]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [52]:
me == x_ptr.owner

True

Dan akhirnya, sama seperti kita dapat memanggil .send() di sebuah tensor, we juga dapat memanggil .get() di sebuah *pointer* untuk sebuah tensor untuk memperoleh tensor itu kembali!!

In [53]:
x_ptr

(Wrapper)>[PointerTensor | me:43091033991 -> bob:80505619433]

In [54]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [55]:
y_ptr

(Wrapper)>[PointerTensor | me:71376756070 -> bob:16737735861]

In [56]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [57]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [58]:
bob._objects

{}

Dan seperti yang anda lihat... Bob tidak punya tensor apapun lagi!! Mereka kembali ke mesin kita!

# Bab 1.2 - Menggunakan *Pointer* Tensor

Mengirim dan menerima tensor dari Bob memang keren, tapi ini bukanlah Pembelajaran Dalam! Kami ingin anda dapat melakukan _operations_ tensor di tensor yang jauh. Untungnya, *pointer* tensor mempermudah hal ini! Anda dapat menggunakan *pointer* seperti tensor normal anda!

In [59]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [60]:
z = x + y

In [61]:
z

(Wrapper)>[PointerTensor | me:94856264424 -> bob:28561975221]

Voilà! 

Di belakang layar, sesuatu yang sangat kuat terjadi. Alih-alih menghitung penjumlahan x dan y di lokal, perintah diserialisasi dan dikirim ke Bob, yang melakukan komputasi, membuat sebuah tensor z, dan lalu mengembalikan pointer z ke kita!

Jika kita memanggil .get() di *pointer*, kita akan menerima hasilnya di mesin kita!

In [62]:
z.get()

tensor([2, 3, 4, 5, 6])

### Fungsi-Fungsi Torch

API ini dikembangkan untuk semua operasi Torch!!!

In [63]:
x

(Wrapper)>[PointerTensor | me:74208622935 -> bob:43188411715]

In [64]:
y

(Wrapper)>[PointerTensor | me:35512884634 -> bob:65201623471]

In [65]:
z = torch.add(x,y)
z

(Wrapper)>[PointerTensor | me:64176255024 -> bob:89886033678]

In [66]:
z.get()

tensor([2, 3, 4, 5, 6])

### Variabel (termasuk *backpropagation!)

In [67]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [68]:
z = (x + y).sum()

In [69]:
z.backward()

(Wrapper)>[PointerTensor | me:27916487465 -> bob:61790140014]

In [70]:
x = x.get()

In [71]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [72]:
x.grad

tensor([1., 1., 1., 1., 1.])

Seperti yang anda dapat saksikan, API ini sangat fleksibel dan mampu melakukan operasi apapun yang anda ingin lakukan di Torch pada *remote data*. Inilah dasar dari protokol penjagaan privasi tingkat lanjut seperti    *Federated Learning, Secure Multi-Party Computation, dan Differential Privacy* !

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

# Selamat!!! - Saatnya Bergabung ke Komunitas

Selamat karena telah menyelesaikan tutorial ini! Jika anda menikmatinya dan ingin bergabung dengan gerakan menuju pertahanan privasi, kepemilikan terdesentralisasi dari AI dan rantai suplai AI (data), anda dapat melakukannya dengan berbagai cara berikut!

### Bintangi PySyft di GitHub

Cara termudah untuk membantu komunitas ini adalah hanya dengan memberikan bintang di repo GitHub! Hal ini membantu meningkatkan kesadaran akan peralatan keren yang lagi dibuat.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Bergabung dengan Slack kami!

Cara terbaik untuk tetap terkini akan kemajuan termutakhir adalah dengan bergabung ke komunitas kami! Anda dapat mengisi form berikut
[http://slack.openmined.org](http://slack.openmined.org)

### Bergabung dengan Project Kode!

Cara terbaik untuk berkontribusi ke komunitas ini adalah dengan menjadi kontributor kode! Kapanpun anda dapat pergi ke laman PySyft GitHub Issues dan filter "Projects". Ini akan memperlihatkan anda gambaran umum Tickets terhadap suatu proyek yang akan anda ikut serta! Jika anda tidak ingin bergabung suatu proyek, tapi ini mengoding sedikit, anda dapat juga mencari mini-proyek dengan mencari di GitHub issues tanda "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donasi

Jika anda tidak memiliki waktu untuk berkontribusi ke basis kode kami, tapi tetap ingin memberikan dukungan, anda dapat juga menjadi Backer di Open Collective kami. Semua donasi akan ditujukan ke hosting web dan pengeluaran komunitas yang lain seperti hackathon dan meetup!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)